# Install required frameworks (only once for all)

In [ ]:
!pip install -U sentence_transformers llama-index llama-index-llms-ollama llama-index-embeddings-huggingface llama-index-vector-stores-deeplake llama-index

## If you are lucky enough to be in the list of sudoer's:

In [ ]:
!sudo curl -fsSL https://ollama.com/install.sh | sh
!ollama pull llama3

## But you are most likely out of sudoer's list and the kernel does not allow you to run processes in background. 
- So, copy/paste the following commands in your terminal
- you can change 'llama3' to any LLM supported bu Ollama
- After each logout/login, you only need to restart the server with ```./bin/ollama serve &```

```
wget https://github.com/ollama/ollama/releases/download/v0.3.9/ollama-linux-amd64.tgz
tar xvfz ollama-linux-amd64.tgz
./bin/ollama serve &
./bin/ollama pull llama3
```

# Configuration (start here for each run)

In [ ]:
# Search for the followind terms in patents
terms = ["medical", "device"]
# pick a relevant embedding model from Hugginface
model_name="NeuML/pubmedbert-base-embeddings-matryoshka"
# Search configuration for EPAB client
match_all = False # Should patents match all terms ?
ignore_case = True # Should we ignore character case during the search
# PROD : >7M publications
# TEST : ~10K publications
epab_size = "PROD" # choose database
# max number of patents to be stored and indexed for RAG
limit = 50
# name of index file
index_dirname = "./storage"
!mkdir -p ./storage
# name of LLM
llm="llama3"
# max number of tokens in prompt
token_limit = 3000

# Fetch the patent data and store it to disk

In [ ]:
!mkdir -p data
!rm -f data/*
from bs4 import BeautifulSoup
from tqdm import tqdm
from epo.tipdata.epab import EPABClient

epab = EPABClient(env=epab_size)
q = epab.query_description(text=",".join(terms), match_all=match_all, ignore_case=ignore_case)
all = "all" if match_all else "one of the"
print(f"Found {q} publications containing the {all} following terms: {terms}")
tab = q.get_results(
    "epab_doc_id, title.fr, abstract, description, publication, inventor", limit=limit
)
print(f"Storing {limit} patents to disk...")
for offset in tqdm(range(limit)):
    data = tab["description.text"][offset]
    with open("data/" + tab["epab_doc_id"][offset] + ".txt", "w") as file:
        print(data, file=file)
    soup = BeautifulSoup(data, "html.parser")
print("Done.")

# Now :
- let's vectorise patents and build the index
- Then, start asking questions about these patents

In [ ]:
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.vector_stores.deeplake import DeepLakeVectorStore

# construct vector store and customize storage context
vector_store = DeepLakeVectorStore(dataset_path=index_dirname)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Load documents and build index
documents = SimpleDirectoryReader("data").load_data()

# embedding model
Settings.embed_model = HuggingFaceEmbedding(
    model_name=model_name
)

# ollama
Settings.llm = Ollama(model=llm, request_timeout=360.0)
print("Indexing patents can take some time...")
index = VectorStoreIndex.from_documents(
    documents, show_progress=True, storage_context=storage_context
)

print("Indexing patents completed...")
!rm -f data/*

# Basic Question Answering loop (no memory of previous questions) 
As patents are stored in datalake, this notebook block can be stopped and restarted as wanted 

In [ ]:
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core import StorageContext

# define vector store
vector_store = DeepLakeVectorStore(dataset_path=index_dirname)

print(f'Loading index from {index_dirname}')
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

query_engine = index.as_query_engine()
while True:
    query = input("How can I help ? (answer 'bye' to quit) " + "\n>")
    if query == "bye":
        break
    response = query_engine.query(query)
    print(response)

# Chat with Llama3 about patents
Use the IPython UI

In [ ]:
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.vector_stores.deeplake import DeepLakeVectorStore

from IPython.display import display
import ipywidgets as widgets

import warnings
import os

warnings.filterwarnings('ignore')

# embedding model
Settings.embed_model = HuggingFaceEmbedding(
    model_name=model_name
)
# ollama
Settings.llm = Ollama(model=llm, request_timeout=360.0)

print(f'Loading index from {index_dirname}')
index = VectorStoreIndex.from_vector_store(vector_store=vector_store,  show_progress=True)
print('Task Completed')
memory = ChatMemoryBuffer.from_defaults(token_limit=token_limit)
chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        " about patents. Do not invent patent numbers"
    ),
)

# Create the interactive widgets
input_box = widgets.Text(
    placeholder='Type your question here',
    description='Question:',
    disabled=False
)

output_area = widgets.Output()

def on_ask_button_click(b):
    with output_area:
        question = input_box.value
        response = chat_engine.chat(question)
        print(response)
        
ask_button = widgets.Button(
    description='Ask',
    disabled=False,
    button_style='',
    tooltip='Ask the question',
    icon='check'
)

def on_reset_button_click(b):
    with output_area:
        output_area.clear_output()
        chat_engine.reset()

reset_button = widgets.Button(
    description='Reset',
    disabled=False,
    button_style='',
    icon='refresh'
)

ask_button.on_click(on_ask_button_click)
reset_button.on_click(on_reset_button_click)
display(output_area, input_box, ask_button, reset_button)